# Thinking 1 ：什么是近似最近邻查找，常用的方法有哪些

答：  
一、近邻算法有两种：KNN（K - Nearest Neighbor） 与 ANN（approximate - Nearest Neighbor），相比于 KNN 精确但复杂度高的查找方式（计算所有距离取前 k 个），ANN 使用近似的距离查找邻居，牺牲一定的精度但时间复杂度更低。ANN 即为近似最近邻查找。

二、近似最近邻查找常用的方法有：LSH（局部敏感 Hash）、LSHForest、LSHensembel、SimHash

# Thinking 2 ：为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度

答：  
一、设两列中元素有以下几种关系：
>1. a : (1, 1)
2. b : (1, 0)
3. c : (0, 0)  

二、Jaccard 相似度：交比并
-->> $ \frac{a}{a+b} $

三、MinHash： 每个个体第一次出现（1）这个情况的行号  

四、重复采样 m 次后（变换 m 次后），两列 MinHash 相等的概率：   
>1. 行特征值的多次重复随机（变换行号，每一列出现在第一列的概率相同，充分考虑每一个特征在第一列的情况，如果 m 足够大，则可以遍历所有 a，b 的可能性），  
2. 两列相等的概率，就等于第一列相等（1，1）的概率，因为MinHash 不考虑（0，0）情况，第一列相等的概率为 (1,1) / [(1,1) + (1,0) + (0,1)]：$ \frac{a}{a+b} $  

这样，就通过概率值将 Jaccard 相似度转化为了近似的概率问题（m 次重复实验，两列 MinHash 相等的概率）并且为了满足降维的需求，这里的 m 一定小于原特征数量。

# Thinking 3 ：SimHash在计算文档相似度的作用是怎样的？ 

答：  
一、使用的相似度计算规则：汉明距离 -->> 两个二进制串中不同位置的数量（汉明距离越小，相似度越高）  

二、SimHash 的编码规则：
>1. 首先对文档进行分词，得到文档特征（N- Gram 等）；
2. 初始化 SimHash 编码位数为 0 ；
3. 对每篇文档的特征计算其重要程度（即为权重 w）；
4. 对每篇文档的特征使用 hash 函数编码成二进制串（SimHash相同位数）；
5. 对每篇文档特征的二进制串与权重相乘，规则为 0 代表 -1 、1 代表 1 ，得到每个特征的编码；
6. 对每篇文档特征的编码对应位置求和，并将负值转化为 0 ，正值转化为 1 ，即为每篇文档最后的 SimHash 编码；

三、对于大数据集，需要进行降维：
>1. 抽屉原理：一般我们认为汉明距离在 3 以内即为相似的，因此将编码分为 4 类，则相似的文档至少有一类是相似的；
2. 对四类中的文档进行精确的匹配（相等概率为$1/2^{16}$），得到相应的候选集：10亿数据集大约会得到 100 万（$2^{34-16}$）的候选集

四、使用汉明距离对候选集中的每篇文档的 SimHash 二进制编码进行相似度计算。  


# Thinking 4 ：为什么YouTube采用期望观看时间作为评估指标

答：  
取决于商业目标：  
因为 YouTube 是一个视频网站，用户喜欢一个视频大概率会观看时间更长，所以不能仅仅使用 CTR 来进行预测，需要进行 观看时间 作为评估指标。

# Action 1 ：使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt
>针对某句话进行Query，查找Top-3相似的句子

In [3]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import jieba.posseg as pseg
import re

#文件读取
f = open('./weibos.txt', 'r', encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号
weibos = re.split('[。！？]', text.replace('\n', '。'))
#print(weibos)

# 最后一行如果为空，则删除
weibos = [weibo for weibo in weibos if weibo != ""]

with open('chinese_stopwords.txt', 'r', encoding = 'utf-8') as file:
    #去掉换行符
    stop_words = [i[:-1]for i in file.readlines()]
# print(weibos)

#分词
def get_corpus(words):
    corpus = str()
    for word in list(words):
        if word.word not in stop_words:
            corpus += str(word.word + ' ')
    return corpus


def get_all_corpus(weibos):
    all_corpus = []
    for weibo in weibos:
        words = pseg.cut(weibo)
        corpus = get_corpus(words)
        all_corpus.append(corpus)
    all_corpus = [corpus for corpus in all_corpus if corpus != '']
    return all_corpus


#创建MinHash
def get_mh(corpus):
    mh = MinHash()
    for word in corpus:
        mh.update(word.encode('utf-8'))
        return mh

def get_forest(all_corpus):
    forest = MinHashLSHForest()
    minhash_list = []
    i = 0
    for corpus in all_corpus:
        mh = get_mh(corpus)
        minhash_list.append(mh)
        forest.add(i, mh)
        i += 1
    return forest, minhash_list

jieba.add_word("里皮")
jieba.add_word("国足")
all_corpus = get_all_corpus(weibos)
# print(all_corpus)
forest, minhash_list = get_forest(all_corpus)
# print(len(minhash_list), len(all_corpus))


#建立索引
forest.index()

query = "国足输给叙利亚后，里皮坐不住了，直接辞职了"
print("查找句：({}) 的相似句".format(query))
query_corpus = get_corpus(pseg.cut(query))
# print(query_corpus)
query_mh = get_mh(query_corpus)

results = forest.query(query_mh, 3)
print(results)
for result in results:
    print('相似句{}：{} \n 与原句相似度：{}'.format(result, all_corpus[result], query_mh.jaccard(minhash_list[result])))

查找句：(国足输给叙利亚后，里皮坐不住了，直接辞职了) 的相似句
[2, 42, 46]
相似句2：国足 输给 叙利亚 之后 里皮 辞职  
 与原句相似度：1.0
相似句42：国足 输给 叙利亚 里皮 坐不住 直接 辞职  
 与原句相似度：1.0
相似句46：国足 昨晚 输给 叙利亚 赛后 主帅 里皮 宣布 辞职  
 与原句相似度：1.0


In [3]:
# with open('chinese_stopwords.txt', 'r', encoding = 'utf-8') as file:
#     stopwords = [i[:-1]for i in file.readlines()]#去掉换行符  \n
# print(stopwords)

# 本章任务

In [3]:
import xlrd
data = xlrd.open_workbook('L6-2自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-6):
    print(table.row(row_index)[:2])
for row_index in range(nrows-5, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

27
[text:'原理', text:'什么是近似最近邻查找，常用的方法有哪些']
[text:'原理', text:'什么是Hash函数']
[text:'原理', text:'什么是k-shingle']
[text:'原理', text:'最小哈希值']
[text:'原理', text:'MinHash执行']
[text:'原理', text:'MinHashLSH']
[text:'原理', text:'MinHash与Jaccard相似度']
[text:'工具', text:'DataSketch中的MinHash, MinHashLSH, MinHashLSHForest, MinHashLSHEnsemble']
[text:'工具', text:'如何使用MinHashLSHForest对新闻句子Top-K相似内容进行查找']
[text:'原理', text:'汉明（Hamming）距离']
[text:'原理', text:'SimHash算法流程']
[text:'原理', text:'如何通过SimHash算法得到每篇文档指纹']
[text:'工具', text:'如何通过SimHash工具计算两篇文档的Hamming距离']
[text:'原理', text:'YouTube召回阶段DNN（输入、隐藏层、输出）']
[text:'原理', text:'YouTube排序阶段DNN（输入、隐藏层、输出）']
[text:'原理', text:'正负样本和上下文选择']
[text:'原理', text:'负采样 Negative Sampling']
[text:'原理', text:'为什么YouTube采用期望观看时间作为评估指标']
[text:'原理', text:'为什么YouTube在排序阶段没有采用经典的LR（逻辑回归）当作输出层，而是采用了Weighted Logistic Regression？']
[text:'Thinking1', text:'什么是近似最近邻查找，常用的方法有哪些'] text:'能简要说明近似最近邻查找（5point）\n常用的方法（5point）'
[text:'Thinking2', text:'为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度'] tex